<a href="https://colab.research.google.com/github/Aayushktyagi/Pytorch_resources/blob/master/torchAutoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Notebook conatins following
Loading mnist data 

*   Loading Mnist dataset 
*   Create Autoencoder 
*   Visualize predictions 




In [ ]:
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotplib.pyplot as plt
import torch.nn as nn
import numpy as np


In [ ]:
# Hyper parameters
latent_dim = 64
batch_size = 128
LR = 0.005
Download_minist = False
epochs = 100

In [ ]:
# Load dataset 
dataset = dset.MNIST(root = './data',
                     train = True,
                     transforms = transforms.ToTensor(),
                     download = Download_mnist)

data_loader = torch.utils.data.DataLoader(dataset = dataset , 
                                          batch_size = batch_size,
                                          shuffle = True)

In [ ]:
# to cuda

def to_cuda(x):
  if torch.cuda.is_available():
    x = x.cuda()
  return Variable(x)




In [ ]:
# Define model

class Autoencoder(nn.Module):
  def __init__(self , in_dim = 784 , h_dim = 400):
    super(Autoencoder ,self).__init__()

    self.encoder = nn.Sequential(
        nn.Linear(in_dim , h_dim),
        nn.ReLU()
    )

    self.decoder = nn.Sequential(
        nn.Linear(h_dim , in_dim)
        nn.Sigmoid()
    )

  def forward(self , x):
    out = self.encoder(x)
    out = self.decoder(out)

    return out 

ae = Autoencoder(in_dim = 784 , h_dim = latent_dim)

if cuda.torch.is_available():
  ae.cuda()


In [ ]:
# loss
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(ae.parameters() , lr = LR)


In [ ]:
# training 
